# Loop transfer recovery


Designing a controller with the linear quadratic regular (LQR) framework not 
only has many advantages in the time-domain but also very good properties in 
the frequency-domain. In particular the gain and phase margins are extremly 
large, $(\frac{1}{2},\infty)$ and $(-\frac{\pi}{3},\frac{\pi}{3})$, respectively.

However, as a famous paper by John Doyle (Guaranteed Margins for LQG control, 
IEEE Automatic Control,1978, n.4, 23) showed the margins for a general LQG design, 
i.e., when only partial state information is available for feedback, can be 
arbitrarily poor.

It is important, however, to note that such an example considers arbitrarily 
chosen values for the parameters defining the LQG problem, i.e., the covariance 
matrices of the gaussian noise and state disturbances and the cost function 
matrices weighting the state and the input. We can actually choose such matrices 
in such a way that we recover the guaranteed phase and gain margins of the LQR 
even even only partial information is available (LQG problem).

This live script illustrates this by considering the original example provided 
in the paper by John Doyle, showing these arbitrarily poor gain and phase margins, 
and then using for the same example the LTR method, showing that the margins 
can be made very large.

To see these fact, run the following script with
```
optphasemargins = 1
```
(Doyle's example) and with
```
optphasemargins = 2
```
(LTR parameters).

In [ ]:
import numpy as np
import control
control.use_numpy_matrix(False)
from course_functions import kalman

In [ ]:
optphasemargins = 1 #1- poor (Doyle), 2-good(LTR)

A = np.array([[1, 1],[0, 1]])
B = np.array([[0],[1]])
C = np.array([1, 0])
n = 2
W = np.ones((2,2))
V = 1

if optphasemargins == 1:
    Q = 1*np.array([1, 1]).reshape(2,1)@np.array([1, 1]).reshape(1,2)
    R = 0.001
else:
    Q = C.T@C
    R = 1e-10

# obtain controller and observer gains of the optimal lqg compensator
K,S,E = control.lqr(A,B,Q,np.array(R))
K = -K
L, Theta = kalman(A,np.eye(n),C,W,V)[:2]
# transfer function of the lqg controller
tflqg = control.ss2tf( A+B@K-L*C ,L,-K,0)
numLQG = tflqg.num[0]
denLQG = tflqg.den[0]
# transfer function of the plant
tfplant = control.ss2tf( A , B , C ,0)
numplant = tfplant.num[0]
denplant = tfplant.den[0]

# nyquist plot taking into accoun open loop transfer function tfplant*tflqg
control.nyquist(tfplant*tflqg, arrowhead_length=0.05, arrowhead_width=0.05);